<a href="https://colab.research.google.com/github/santiagogutierrezreina/IA/blob/main/Clasificador_de_Imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importamos librerias
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#Obtenemos el dataset de fashion mnist con las 60.000 imagenes de entrenamiento y 10.000 de test
datos, metadatos = tfds.load('fashion_mnist', as_supervised=True, with_info=True)

In [ ]:
metadatos

In [ ]:
#Separamos los datos de entrenamiento y prueba
datos_entrenamiento = datos['train']
datos_pruebas = datos['test']

In [ ]:
#Obtenemos las etiquetas
nombres_clases = metadatos.features['label'].names

In [ ]:
nombres_clases

In [ ]:
#Normalizamos los datos entre 0-1
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32) #Transforma imagenes a un dato tipo float32
  imagenes /= 255                          #La divide entre 255 para normalizarla de 0-255 a 0-1
  return imagenes, etiquetas

#Normalizamos los datos de entrenamiento y test
datos_entrenamiento = datos_entrenamiento.map(normalizar) #.map aplica la funcion interna a cada elemento del dataset
datos_pruebas = datos_pruebas.map(normalizar)

#Agregamos a cache(memoria en lugar de disco)
datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

In [ ]:
#Mostrar una imagen de los datos de prueba
for imagen, etiqueta in datos_entrenamiento.take(1): #De momento, solo se imprime la primer imagen
  break
imagen = imagen.numpy().reshape((28,28)) #Redimensionamos la imagen a 28x28

#Dibuja la imagen asi, ay que bonito
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(imagen, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)

In [ ]:
plt.figure(figsize=(10,10)) #Crea una figura de 10"x 10"
for i , (imagen, etiqueta) in enumerate(datos_entrenamiento.take(25)): #Crea un contador con los 25 datos
  imagen = imagen.numpy().reshape((28,28))
  plt.subplot(5,5,i+1) #Crea una matriz 5x5 donde se almacena cada imagen creada con su etiqueta

  #Limpiamos ticks y grilla
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)

  #Muestra la imagen en la escala de grises y la etiqueta en el xlabel
  plt.imshow(imagen, cmap=plt.cm.binary)
  plt.xlabel(nombres_clases[etiqueta])
plt.show()

In [ ]:
#Creamos el modelo
modelo = tf.keras.Sequential([
    #Creamos la capa entrada (Flatten "aplasta" a una sola dimension)
    tf.keras.layers.Flatten(input_shape=(28,28,1)), #1 - blanco y negro

    #Creamos las capas ocultas densas, con 50 neuronas y la funcion reLU
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),

    #softmax se emplea para aplicar la funcion de activacion en la capa de salida, para las redes de clasificacion, asegurando que siempre la salida sea 1
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) #10 - numero de clases
])

In [ ]:
#Compilamos el modelo
modelo.compile(
    optimizer='adam', #Optimizador
    loss='sparse_categorical_crossentropy', #Funcion de perdida
    metrics=['accuracy'] #Metrica
)

In [ ]:
#Guardamos en variables la cantidad de datos de entrenamiento y pruebas
num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples

In [ ]:
print(num_ej_entrenamiento)
print(num_ej_pruebas)

In [ ]:
#Definimos en lotes para no sobrecargar el modelo con la cantidad de datos de prueba
TAMANO_LOTE = 32

#Separamos los datos en los batches o lotes
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE) #Repetimos los datos de entrenamiento de forma aleatoria
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

In [ ]:
import math

#Entrenamos el modelo con 5 vueltas, idealmente debe llegar a nna precision del 88%
historial = modelo.fit(datos_entrenamiento, epochs=5, steps_per_epoch=math.ceil(num_ej_entrenamiento/TAMANO_LOTE)) #steps-per-epoch va a ser removida, por ende no es relevante

In [ ]:
#Graficamos la funcion de perdida
plt.xlabel("# Epoca")
plt.ylabel("Magnitud de pérdida")
plt.plot(historial.history["loss"])

In [ ]:
#Pintar una cuadricula con varias predicciones, y marcar si fue correcta (azul) o incorrecta (roja)
import numpy as np

for imagenes_prueba, etiquetas_prueba in datos_pruebas.take(1):
  imagenes_prueba = imagenes_prueba.numpy()
  etiquetas_prueba = etiquetas_prueba.numpy()
  predicciones = modelo.predict(imagenes_prueba)

def graficar_imagen(i, arr_predicciones, etiquetas_reales, imagenes):
  arr_predicciones, etiqueta_real, img = arr_predicciones[i], etiquetas_reales[i], imagenes[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  etiqueta_prediccion = np.argmax(arr_predicciones)
  if etiqueta_prediccion == etiqueta_real:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(nombres_clases[etiqueta_prediccion],
                                100*np.max(arr_predicciones),
                                nombres_clases[etiqueta_real]),
                                color=color)

def graficar_valor_arreglo(i, arr_predicciones, etiqueta_real):
  arr_predicciones, etiqueta_real = arr_predicciones[i], etiqueta_real[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  grafica = plt.bar(range(10), arr_predicciones, color="#777777")
  plt.ylim([0, 1])
  etiqueta_prediccion = np.argmax(arr_predicciones)

  grafica[etiqueta_prediccion].set_color('red')
  grafica[etiqueta_real].set_color('blue')

filas = 5
columnas = 5
num_imagenes = filas*columnas
plt.figure(figsize=(2*2*columnas, 2*filas))
for i in range(num_imagenes):
  plt.subplot(filas, 2*columnas, 2*i+1)
  graficar_imagen(i, predicciones, etiquetas_prueba, imagenes_prueba)
  plt.subplot(filas, 2*columnas, 2*i+2)
  graficar_valor_arreglo(i, predicciones, etiquetas_prueba)

In [ ]:
#Tomar cualquier dato para realizar una prediccion
imagen = imagenes_prueba[0]
imagen = np.array([imagen])
prediccion = modelo.predict(imagen)

print("Prediccion: " + nombres_clases[np.argmax(prediccion[0])])